In [1]:
import mxnet as mx
import logging
import numpy as np
import os, urllib2, time
import warnings
warnings.simplefilter('ignore',DeprecationWarning)

In [2]:
import pickle as pkl

data = pkl.load(open('data/svhn_64.p', 'r'))

In [3]:
import numpy as np
from numpy import newaxis

data['X_train'] = mx.nd.array(data['X_train'], dtype=np.float32)
data['X_test'] = mx.nd.array(data['X_test'], dtype=np.float32)
data_x_train = data['X_train'].reshape((-1,3,64,64))
data_x_valid = data['X_test'].reshape((-1,3,64,64)) 
data['y_train'] = mx.nd.array(data['y_train'])
data['y_test'] = mx.nd.array(data['y_test'])
data_y_train = data['y_train'] 
data_y_valid = data['y_test'] 
train_set = mx.io.NDArrayIter(data_x_train, data_y_train, batch_size=128)
valid_set = mx.io.NDArrayIter(data_x_valid, data_y_valid, batch_size=128)

In [4]:
print train_set.provide_label
print train_set.provide_data

[DataDesc[softmax_label,(128, 4L),<type 'numpy.float32'>,NCHW]]
[DataDesc[data,(128, 3L, 64L, 64L),<type 'numpy.float32'>,NCHW]]


In [5]:
data = mx.symbol.Variable(name="data")
label = mx.sym.Variable('softmax_label')
conv1_1 = mx.symbol.Convolution(data=data, kernel=(3, 3), pad=(1, 1), num_filter=32, name="conv1_1")
relu1_1 = mx.symbol.Activation(data=conv1_1, act_type="relu", name="relu1_1")
conv1_2 = mx.symbol.Convolution(data=relu1_1, kernel=(3, 3), pad=(1, 1), num_filter=32, name="conv1_2")
relu1_2 = mx.symbol.Activation(data=conv1_2, act_type="relu", name="relu1_2")
pool1 = mx.symbol.Pooling(data=relu1_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool1")
dropout1 = mx.symbol.Dropout(data=pool1, p=0.5,name="dropout1")

conv2_1 = mx.symbol.Convolution(data=dropout1, kernel=(3, 3), pad=(1, 1), num_filter=64, name="conv2_1")
relu2_1 = mx.symbol.Activation(data=conv2_1, act_type="relu", name="relu2_1")
conv2_2 = mx.symbol.Convolution(data=relu2_1, kernel=(3, 3), pad=(1, 1), num_filter=64, name="conv2_2")
relu2_2 = mx.symbol.Activation(data=conv2_2, act_type="relu", name="relu2_2")
pool2 = mx.symbol.Pooling(data=relu2_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool2")
dropout2 = mx.symbol.Dropout(data=pool2, p=0.5,name="dropout2")

conv3_1 = mx.symbol.Convolution(data=dropout2, kernel=(3, 3), pad=(1, 1), num_filter=128, name="conv3_1")
relu3_1 = mx.symbol.Activation(data=conv3_1, act_type="relu", name="relu3_1")
conv3_2 = mx.symbol.Convolution(data=relu3_1, kernel=(3, 3), pad=(1, 1), num_filter=128, name="conv3_2")
relu3_2 = mx.symbol.Activation(data=conv3_2, act_type="relu", name="relu3_2")
flatten = mx.symbol.Flatten(data = relu3_2, name = "flatten")
linear1 = mx.symbol.FullyConnected(data= flatten ,num_hidden=4, name='linear1')
pred = mx.sym.LinearRegressionOutput(data=linear1, label=label, name='lro')


In [6]:
model = mx.mod.Module(pred, context=mx.cpu(0))

In [7]:


from VisCostCallback import CostVisCallback

callbacks = CostVisCallback(nepochs=4.0, y_range=None, total_batches=52)

Loading BokehJS ...

In [8]:
model.fit(
        train_data          = train_set,
        eval_data           = valid_set,
        eval_metric         = mx.metric.MSE(),
        optimizer           = 'sgd',
        optimizer_params    = {'learning_rate': 10**-7, 'momentum': 0.9},
        initializer         = mx.init.Xavier(),
        num_epoch           = 4,
        batch_end_callback  = callbacks)

In [9]:
metric = model.score(valid_set, mx.metric.MSE())

In [10]:
model.predict(valid_set, 1).asnumpy()

array([[  25.8197937 ,   18.51831627,   45.00294876,   43.50669861],
       [  19.51347351,   23.45105553,   23.98363113,   27.3955822 ],
       [  10.71810055,    1.04164922,   45.50121689,   50.40250778],
       [  21.2573185 ,    5.01581478,   81.5424118 ,   91.23922729],
       [  26.48350334,   20.78497124,   33.1713562 ,   41.09690857],
       [  22.67494011,   12.39291191,   68.50933075,   77.44200134],
       [  44.69762802,   22.88648415,  119.03982544,  134.09480286],
       [  21.3484478 ,    9.64858913,   69.42225647,   81.39300537],
       [   7.80810356,    4.54313374,   40.37704849,   46.54961777],
       [  18.42778587,   14.01025581,   54.86844635,   59.26881409],
       [  11.17209721,    8.57215595,   20.66305161,   19.27395058],
       [  21.52748299,    9.69792366,   62.02340698,   73.99660492],
       [  28.89739418,   27.24561691,   39.59315109,   43.24203873],
       [  19.06996727,    6.27894735,   67.20481873,   77.54160309],
       [  23.05193329,    9.966922

In [11]:
valid_set.getdata()[0].asnumpy()

array([[[[  98.,   99.,  101., ...,   81.,   83.,   83.],
         [  98.,  100.,  101., ...,   80.,   83.,   85.],
         [  98.,  100.,  101., ...,   80.,   85.,   85.],
         ..., 
         [ 121.,  122.,  122., ...,  124.,  126.,  124.],
         [ 116.,  115.,  115., ...,  131.,  129.,  125.],
         [ 109.,  109.,  108., ...,  134.,  132.,  128.]],

        [[  88.,   89.,   91., ...,   72.,   72.,   73.],
         [  88.,   90.,   91., ...,   73.,   74.,   74.],
         [  88.,   90.,   91., ...,   73.,   76.,   76.],
         ..., 
         [ 112.,  113.,  113., ...,  117.,  119.,  117.],
         [ 107.,  106.,  106., ...,  122.,  120.,  116.],
         [ 100.,  100.,   99., ...,  125.,  123.,  119.]],

        [[  78.,   79.,   81., ...,   65.,   66.,   64.],
         [  78.,   80.,   81., ...,   67.,   67.,   68.],
         [  76.,   78.,   79., ...,   67.,   69.,   67.],
         ..., 
         [ 105.,  106.,  106., ...,  111.,  113.,  111.],
         [  98.,   97.,

In [12]:
T = valid_set.getlabel()[0].asnumpy()